In [1]:
### Initial design of drawing a coplanar waveguide resonator 


# automatic reloading of modules when they change

%load_ext autoreload
%autoreload 2


In [2]:
# Import statements

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.analyses.quantization import EPRanalysis

In [3]:
# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
design.chips.main.size['size_z'] = '-500um'
design.chips.main.size['sample_holder_bottom'] = '500um'
design.chips.main.size['sample_holder_top'] = '2000um'


# Set hfss as renderer for design
hfss = design.renderers.hfss

In [4]:
# Allow overwriting
gui = MetalGUI(design)
design.overwrite_enabled = True
design.chips.main
hfss.default_options

{'Lj': '10nH',
 'Cj': 0,
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'x_buffer_width_mm': 0.2,
 'y_buffer_width_mm': 0.2,
 'wb_threshold': '400um',
 'wb_offset': '0um',
 'wb_size': 5,
 'plot_ansys_fields_options': {'name': 'NAME:Mag_E1',
  'UserSpecifyName': '0',
  'UserSpecifyFolder': '0',
  'QuantityName': 'Mag_E',
  'PlotFolder': 'E Field',
  'StreamlinePlot': 'False',
  'AdjacentSidePlot': 'False',
  'FullModelPlot': 'False',
  'IntrinsicVar': "Phase='0deg'",
  'PlotGeomInfo_0': '1',
  'PlotGeomInfo_1': 'Surface',
  'PlotGeomInfo_2': 'FacesList',
  'PlotGeomInfo_3': '1'}}

In [23]:
# Create shorted pin and open pin for resonator

otg1 = OpenToGround(design, 'open_to_ground1', options=dict(pos_x='0um', pos_y='-0.95mm',
orientation='180'))
stg = ShortToGround(design, 'short_to_ground', options=dict(pos_x='0um', pos_y='0.8mm',
orientation='270')) 


# Create meandered resonator between shorted and opened pin
rt_meander = RouteMeander(design, 'readout', Dict(
        total_length='8 mm',
        hfss_wire_bonds = False,
        fillet='70 um',
        lead = Dict(
        start_straight='0.1mm',
        end_straight='0mm'),
        #start_jogged_extensions=jogs2,
        #end_jogged_extension=jogs),
        meander=Dict(
        spacing = '150um'),
        pin_inputs=Dict(
            start_pin=Dict(component='short_to_ground', pin='short'),
            end_pin=Dict(component='open_to_ground1', pin='open')), ))

gui.rebuild()
gui.autoscale()


In [ ]:
#Setup the launchpad location and orientation.
launch_options1 = dict(pos_x='-750um', pos_y='0.64mm', orientation='0')

lp = LaunchpadWirebondCoupled(design, 'P4_Q', options = launch_options1)

#stg = ShortToGround(design, 'short_to_ground',dict(pos_x='-500um', pos_y='1mm', orientation='0')) 


#Setup the OpenToGround location and orientation.
launch_options2 = dict(pos_x='0.75mm',  pos_y='0.64mm', orientation='180')

lp2 = LaunchpadWirebondCoupled(design, 'P5_Q', options = launch_options2)


#otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='0.75mm', pos_y='0.64mm',
#orientation='180'))

#After the two QComponents are added to design, connect them with a RouteMeander.
meander_options = Dict(
        total_length='1.5 mm',
        #fillet='200 um',
        #lead = dict(start_straight='100um', end_straight='100um'),
       # meander=Dict(
        #spacing = '600um'),
        pin_inputs=Dict(
            start_pin=Dict(component=lp.name, pin='tie'),
            end_pin=Dict(component=lp2.name, pin='tie')), )

meander = RouteStraight(design, 'feed',options=meander_options)

gui.rebuild()
gui.autoscale()

In [6]:
eig_res = EPRanalysis(design, "hfss")
hfss = eig_res.sim.renderer
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')

In [ ]:
#hfss.connect_ansys(project_path = 'C:\\Users\\2254729M\\Documents\\Ansoft', project_name='Project22')

In [24]:
hfss.connect_ansys()

INFO 06:10PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:10PM [load_ansys_project]: 	Opened Ansys App
INFO 06:10PM [load_ansys_project]: 	Opened Ansys Desktop v2019.1.0
INFO 06:10PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/2254729M/Documents/Ansoft/
	Project:   Project23
INFO 06:10PM [connect_design]: 	Opened active design
	Design:    Readout [Solution type: Eigenmode]
INFO 06:10PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 06:10PM [connect]: 	Connected to project "Project23" and design "Readout" 😀 



In [8]:
hfss.activate_ansys_design("Readout", 'eigenmode') 

06:03PM 35s WARNING [activate_ansys_design]: The design_name=Readout was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 06:03PM [connect_design]: 	Opened active design
	Design:    Readout [Solution type: Eigenmode]
WARNING 06:03PM [connect_setup]: 	No design setup detected.
WARNING 06:03PM [connect_setup]: 	Creating eigenmode default setup.
INFO 06:03PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [ ]:
#hfss.render_design(['short_to_ground', 'readout', 'open_to_ground1', lp.name, lp2.name, 'feed'], [])
hfss.render_design([stg.name, otg1.name, 'readout'],[]) #, lp.name, lp2.name, 'feed'], [])
#hfss.save_screenshot()

In [ ]:
hfss.render_design([stg.name, otg1.name, 'readout'],[])

In [25]:
hfss.render_design(selection=[stg.name, otg1.name, 'readout'],
                   open_pins=[],
                   port_list=[],
                   jj_to_port=[],
                   ignored_jjs=[],
                   box_plus_buffer = False)

In [ ]:
design._chips['main']['size']['size_y'] = '3mm'

In [22]:
hfss.clean_active_design()
#hfss.render_design([], [], box_plus_buffer=False)

In [ ]:
hfss.subtract_from_ground()

In [26]:

setup = hfss.pinfo.setup
setup.passes = 20
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

INFO 06:12PM [analyze]: Analyzing setup Setup



Number of eigenmodes to find             = 1
Number of simulation passes              = 20
Convergence freq max delta percent diff  = 0.1

This method is deprecated. Change your scripts to use the equivalent method plot_convergence() that has been moved inside the EPRanalysis class.


In [28]:

eig_res.sim.convergence_t, eig_res.sim.convergence_f, _ = hfss.get_convergences()
eig_res.sim.plot_convergences()

  df = pd.read_csv(io.StringIO(text2[3].strip()),

06:26PM 12s INFO [get_f_convergence]: Saved convergences to C:\Users\2254729M\OneDrive - University of Glasgow\Documents\workspace\Thesis\Test Designs\hfss_eig_f_convergence.csv
INFO 06:26PM [__del__]: Disconnected from Ansys HFSS


In [29]:
hfss.disconnect_ansys()